In [1]:
from manim import *
from numpy import linalg as npl
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline

video_scene = " -v WARNING --progress_bar None --disable_caching comf_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s comf_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.8


In [70]:
# Integrationsparameter
T_general = 10
dt_general = 0.01

# Parameter
m1 = 5
m2 = 5
red_m = m1*m2 / (m1 + m2) 

G_general = 11
b_general = 0.6

r1_general = np.sqrt(m1/PI/10)
r2_general = np.sqrt(m2/PI/10)
r1_red_general =  np.sqrt(red_m/PI/10)

# Positions und 
pos1_general = np.array([-5., 0.])
pos2_general = np.array([0., pos1_general[1] - b_general])

v1_general = np.array([10., 0.])
v2_general = np.array([0., 0.])

In [71]:
# schiefer Stoß
def push_pos_i(T, dt):
    N = int(T/dt)

    b = b_general
    r1 = r1_general
    r2 = r2_general

    pos1 = pos1_general.copy()
    pos2 = pos2_general.copy()

    v1 = v1_general.copy()
    v2 = v2_general.copy()

    pos1_array = np.zeros((N+1, 3))
    pos2_array = np.zeros((N+1, 3))

    pos1_array[0,:2] = pos1
    pos2_array[0,:2] = pos2

    i = 1

    # Konstante Geschwindigkeit von m1 bis Treffen auf m2
    while (r1 + r2) < npl.norm(pos1 - pos2):
        pos1 += v1 * dt
        pos2 += v2 * dt

        pos1_array[i,:2] = pos1
        pos2_array[i,:2] = pos2

        i += 1

    # Vollkommen elastischer Stoß von m1 auf m2
    theta2 = np.arccos(np.dot(pos2-pos1, v1)/(npl.norm(pos2-pos1)*npl.norm(v1)))
    theta = PI - 2*theta2
    v2 = (pos2 - pos1) / npl.norm(pos2 - pos1) * npl.norm(v1) * 2*m1 / (m1 + m2) * np.sin(theta/2)  
    v1 = np.array([-v2[1], v2[0]]) / npl.norm(v2)  * npl.norm(v1) * np.sqrt(m1**2 + m2**2 + 2*m1*m2*np.cos(theta)) / (m1 + m2)

    t_break = (i - 1, v1, v2)

    # Konstante Geschwindigkeit m1 und m2 bis Ende des Arrays
    while (i <= N):
        pos1 += v1 * dt
        pos2 += v2 * dt

        pos1_array[i,:2] = pos1
        pos2_array[i,:2] = pos2

        i += 1

    return pos1_array, pos2_array, t_break


# Streuung
def scatter_pos_i(T, dt):
    N = int(T/dt)

    G = G_general
    b = b_general

    pos1 = pos1_general.copy()
    pos2 = pos2_general.copy()

    v1 = v1_general.copy()
    pos1_array = np.zeros((N+1, 3))
    pos1_array[0,:2] = pos1-pos2

    i = 1

    while (i <= N):
        r_dist = npl.norm(pos1-pos2)

        # Updaten der Beschleunigung a
        a1 = G * m2 / r_dist**3 * (pos1-pos2)
        #print(a1)

        pos1 += v1 * dt
        pos1_array[i,:2] = pos1-pos2

        # Updaten der Geschwindigkeit v
        v1 += a1 * dt
        
        i += 1

    return pos1_array


# Generierung der Arrays
T = T_general
dt = dt_general

pos1_array_general, pos2_array_general, t_break_general = push_pos_i(T, dt)
pos1_red_array_general = scatter_pos_i(T, dt)

In [17]:
#pos1_red_array_general, pos1_general

(array([[ -5.        ,   0.4       ,   0.        ],
        [ -4.9       ,   0.4       ,   0.        ],
        [ -4.8001981 ,   0.40001585,   0.        ],
        ...,
        [-18.00688252, 104.86247455,   0.        ],
        [-18.02588062, 104.97505407,   0.        ],
        [-18.0448788 , 105.08763403,   0.        ]]),
 array([-5.,  0.]))

In [ ]:
%%capture_video --path "animations/center_of_mass_force/noncentral_zoom.mp4"
%%manim -qm --fps 60 $video_scene

class comf_Scene(MovingCameraScene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        text1 = Text("(a) Laborsystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)

        pos1_array, pos2_array, c_break = pos1_array_general, pos2_array_general, t_break_general
        t_break = c_break[0]
        vgen = np.array([v1_general[0], 0, 0])
        v1_break = np.array([c_break[1][0], c_break[1][1], 0])
        v2_break = np.array([c_break[2][0], c_break[2][1], 0])

        # Achsen zu Masse 2
        cat_x_axis = Line([-50, pos2_array[0][1], 0], [50, pos2_array[0][1], 0], color = WHITE, stroke_width = 0.5)
        cat_y_axis = Line([pos2_array[0][0], -50, 0], [pos2_array[0][0], 50, 0], color = WHITE, stroke_width = 0.5)

        cat_x = Text("x'", font_size = 16).move_to([6.75, pos2_array[0][1] + 0.25, 0])
        cat_y = Text("y'", font_size = 16).move_to([pos2_array[0][0] + 0.25, 3.75, 0])

        # 2 Massen
        m1_dot = Dot(radius = r1, color = BLUE).move_to(pos1_array[0])
        m2_dot = Dot(radius = r2, color = WHITE).move_to(pos2_array[0])

        # Pfeile
        connection_para = Line(pos1_array[t_break], pos2_array[t_break], color = GREY, stroke_width = 0.5)
        vgen_arrow = Line(pos1_array[t_break] - vgen / 10, pos1_array[t_break], color = RED, stroke_width = 2).add_tip(tip_length = 0.1, tip_width = 0.1)
        v1_arrow = Line(pos1_array[t_break], pos1_array[t_break] + v1_break / 10, color = DARK_BLUE, stroke_width = 2).add_tip(tip_length = 0.1, tip_width = 0.1)
        v2_arrow = Line(pos2_array[t_break], pos2_array[t_break] + v2_break / 10, color = DARK_BLUE, stroke_width = 2).add_tip(tip_length = 0.1, tip_width = 0.1)
        arc = ArcBetweenPoints(pos1_array[t_break] + v1_break / 10 - v1_break / npl.norm(v1_break) / 7.5, pos1_array[t_break] + v1_break / 10 + v2_break / npl.norm(v2_break) / 7.5, stroke_width = 1, color = DARK_BLUE)
        arc_c = ArcBetweenPoints(pos1_array[t_break] + v1_break / 10 - v1_break / npl.norm(v1_break) / 10, pos1_array[t_break] + v1_break / 10 + v2_break / npl.norm(v2_break) / 10, stroke_width = 2)
        arc_dot = Dot(arc_c.get_center(), radius = 0.01, color = DARK_BLUE)

        # Pfeilshifts
        vgen_arrow_shift = vgen[0] / 10 * RIGHT 
        v2_arrow_shift = pos2_array[t_break][0] * LEFT + pos2_array[t_break][1] * DOWN + (pos1_array[t_break][0] + v1_break[0] / 10) * RIGHT + (pos1_array[t_break][1] + v1_break[1] / 10) * UP

        # Laborpositionen als Iterator
        m1_dot.iter = iter(pos1_array)
        m2_dot.iter = iter(pos2_array)

        m1_dot.color = BLUE
        m2_dot.color = WHITE

        cat_x_axis.iter = iter(pos2_array)
        cat_y_axis.iter = iter(pos2_array)
        cat_x.iter = iter(pos2_array)
        cat_y.iter = iter(pos2_array)

        traj_group = VGroup()


        # Updater
        def mass_updater(mass):
            next_pos = next(mass.iter)
            con_line = Line(mass.get_center(), next_pos, color = mass.color, stroke_opacity = 1)
            traj_group.add(con_line)
            self.add(con_line)
            mass.move_to(next_pos)

        def cat_x_axis_updater(axis):
            y = next(axis.iter)[1]
            axis.move_to([0, y, 0])

        def cat_y_axis_updater(axis):
            x = next(axis.iter)[0]
            axis.move_to([x, 0, 0])

        def cat_x_updater(label):
            y = next(label.iter)[1] + 0.25
            label.move_to([6.75, y, 0])

        def cat_y_updater(label):
            x = next(label.iter)[0] + 0.25
            label.move_to([x, 3.75, 0])


########## Animation ##########

        self.play(Write(text1), run_time = 1.5)
        self.play(FadeIn(m1_dot), FadeIn(m2_dot), FadeIn(cat_x_axis), FadeIn(cat_y_axis), FadeIn(cat_x), FadeIn(cat_y), run_time = 3)
        self.play(Unwrite(text1), run_time = 1.5)
        self.wait(0.5)

        m1_dot.add_updater(mass_updater)
        m2_dot.add_updater(mass_updater)

        cat_x_axis.add_updater(cat_x_axis_updater)
        cat_y_axis.add_updater(cat_y_axis_updater)
        cat_x.add_updater(cat_x_updater)
        cat_y.add_updater(cat_y_updater)

        # Save the state of camera
        self.camera.frame.save_state()

        time = ValueTracker(0)
        self.play(time.animate.set_value(0), rate_func = linear, run_time = t_break / 60)

        m1_dot.remove_updater(mass_updater)
        m2_dot.remove_updater(mass_updater)

        cat_x_axis.remove_updater(cat_x_axis_updater)
        cat_y_axis.remove_updater(cat_y_axis_updater)
        cat_x.remove_updater(cat_x_updater)
        cat_y.remove_updater(cat_y_updater)

        self.wait(0.5)
        self.play(self.camera.frame.animate.set_height(m1_dot.height * 3), run_time = 1.5)
        self.wait(0.5)

########## END OF START ##########

        self.play(Create(connection_para))
        self.wait(0.5)
        self.play(Create(vgen_arrow), Create(v1_arrow), Create(v2_arrow))
        self.wait(0.5)
        self.play(Uncreate(connection_para))
        self.wait(0.5)
        self.play(ApplyMethod(vgen_arrow.shift, vgen_arrow_shift), ApplyMethod(v2_arrow.shift, v2_arrow_shift))
        self.wait(0.5)
        self.play(Create(arc))
        self.play(Create(arc_dot))
        self.wait(2)
        self.play(Uncreate(arc_dot))
        self.play(Uncreate(arc))
        self.play(ApplyMethod(vgen_arrow.shift, -vgen_arrow_shift), ApplyMethod(v2_arrow.shift, -v2_arrow_shift), FadeOut(vgen_arrow))
        self.wait(0.5)
        #self.play(Uncreate(vgen_arrow), Uncreate(v1_arrow), Uncreate(v2_arrow))

########## FINISH ##########

        self.wait(0.5)
        self.play(self.camera.frame.animate.restore(), run_time = 1.5)
        self.wait(0.5)

        m1_dot.add_updater(mass_updater)
        m2_dot.add_updater(mass_updater)

        cat_x_axis.add_updater(cat_x_axis_updater)
        cat_y_axis.add_updater(cat_y_axis_updater)
        cat_x.add_updater(cat_x_updater)
        cat_y.add_updater(cat_y_updater)

        self.play(FadeOut(v1_arrow), FadeOut(v2_arrow), run_time = 3)
        #self.play(time.animate.set_value(0), rate_func = linear, run_time = 3)

########## R2 ##########

        # R2-Prep
        m1_dot.remove_updater(mass_updater)
        m2_dot.remove_updater(mass_updater)

        cat_x_axis.remove_updater(cat_x_axis_updater)
        cat_y_axis.remove_updater(cat_y_axis_updater)
        cat_x.remove_updater(cat_x_updater)
        cat_y.remove_updater(cat_y_updater)

        m1_dot.iter = iter(pos1_array)
        m2_dot.iter = iter(pos2_array)

        m1_dot.color = BLUE
        m2_dot.color = WHITE

        cat_x_axis.iter = iter(pos2_array)
        cat_y_axis.iter = iter(pos2_array)
        cat_x.iter = iter(pos2_array)
        cat_y.iter = iter(pos2_array)


        # R2
        self.play(
            FadeOut(traj_group), FadeIn(m1_dot.move_to(pos1_array[0])), FadeIn(m2_dot.move_to(pos2_array[0])), 
            FadeIn(cat_x_axis.move_to([0, pos2_array[0][1], 0])), FadeIn(cat_y_axis.move_to([0, pos2_array[0][1], 0])), 
            FadeIn(cat_x.move_to([6.75, pos2_array[0][1] + 0.25, 0])), FadeIn(cat_y.move_to([pos2_array[0][0] + 0.25, 3.75, 0])), run_time = 3)

        self.wait(0.5)

        m1_dot.add_updater(mass_updater)
        m2_dot.add_updater(mass_updater)

        cat_x_axis.add_updater(cat_x_axis_updater)
        cat_y_axis.add_updater(cat_y_axis_updater)
        cat_x.add_updater(cat_x_updater)
        cat_y.add_updater(cat_y_updater)

        time = ValueTracker(0)
        self.play(time.animate.set_value(0), rate_func = linear, run_time = 3)
        self.play(FadeOut(traj_group), run_time = 3)
        #self.wait(3)


In [72]:
%%capture_video --path "animations/center_of_mass_force/scattering.mp4"
%%manim -qm --fps 60 $video_scene

class comf_Scene(MovingCameraScene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        text1 = Text("(a) Laborsystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)

        pos1_red_array = pos1_red_array_general
        pos1_cent2_array = pos1_array_general - pos2_array_general
        pos2_cent2_array = pos2_array_general - pos2_array_general

        r1 = r1_general
        r2 = r2_general
        r1_red = r1_red_general

        # 2 Massen
        m1_red_dot = Dot(radius = r1_red, color = RED, fill_opacity = 0.5).move_to(pos1_red_array[0])
        m1_dot = Dot(radius = r1, color = BLUE).move_to(pos1_cent2_array[0])
        m2_dot = Dot(radius = r2, color = WHITE).move_to(pos2_cent2_array[0])

        m1_red_dot.color = RED
        m1_dot.color = BLUE
        m2_dot.color = WHITE

        # m2-Zentrumskoordinaten als Iterator
        m1_red_dot.iter = iter(pos1_red_array)
        m1_dot.iter = iter(pos1_cent2_array)
        m2_dot.iter = iter(pos2_cent2_array)

        # Updater
        def mass_updater(mass):
            next_pos = next(mass.iter)
            con_line = Line(mass.get_center(), next_pos, color = mass.color, stroke_opacity = 1)
            #traj_group.add(con_line)
            self.add(con_line)
            mass.move_to(next_pos)


        m1_red_dot.add_updater(mass_updater)
        m1_dot.add_updater(mass_updater)
        m2_dot.add_updater(mass_updater)

        self.add(m1_dot, m2_dot, m1_red_dot)

        # Save the state of camera
        time = ValueTracker(0)
        self.play(time.animate.set_value(0), rate_func = linear, run_time = 0.5)
        self.camera.frame.save_state()
        self.play(self.camera.frame.animate.set_height(m1_dot.height * 45), run_time = 3)

        m1_red_dot.remove_updater(mass_updater)
        m1_dot.remove_updater(mass_updater)
        m2_dot.remove_updater(mass_updater)

        self.wait(3)

Output saved by overwring previous file at animations/center_of_mass_force/scattering.mp4.
